In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing

In [2]:
#연간 사망인원이 한 명 이상인 교통사고 데이터

data = pd.read_csv('./datasets/Assignment4(traffic)/도로교통공단_사망 교통사고 정보_20201123.csv', encoding = 'cp949')
data

,발생년,발생년월일시,주야,요일,사망자수,부상자수,중상자수,경상자수,부상신고자수,발생지시도,...,사고유형,가해자법규위반,도로형태_대분류,도로형태,가해자_당사자종별,피해자_당사자종별,발생위치X_UTMK,발생위치Y_UTMK,경도,위도
0,2019,2019-01-01 00,야,화,1,0,0,0,0,부산,...,기타,중앙선 침범,교차로,교차로내,승용차,보행자,1150507.0,1685987.0,129.152465,35.157884
1,2019,2019-01-01 03,야,화,1,0,0,0,0,경기,...,정면충돌,중앙선 침범,단일로,기타단일로,승용차,승용차,943018.0,1921233.0,126.857149,37.288292
2,2019,2019-01-01 16,주,화,1,0,0,0,0,경북,...,측면충돌,안전운전 의무 불이행,교차로,교차로부근,승용차,사륜오토바이(ATV),1100414.0,1739893.0,128.609230,35.650108
3,2019,2019-01-01 19,야,화,1,0,0,0,0,대전,...,횡단중,안전운전 의무 불이행,교차로,교차로부근,승용차,보행자,989194.0,1817621.0,127.379560,36.355946
4,2019,2019-01-01 21,야,화,1,0,0,0,0,경북,...,기타,안전운전 의무 불이행,단일로,기타단일로,승용차,보행자,1127700.0,1767912.0,128.915041,35.899506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,2019,2019-12-31 05,야,화,1,0,0,0,0,서울,...,차도통행중,안전운전 의무 불이행,교차로,교차로부근,화물차,보행자,964440.0,1943029.0,127.097769,37.485819
3229,2019,2019-12-31 08,주,화,1,1,0,1,0,강원,...,추돌,안전운전 의무 불이행,교차로,교차로내,승용차,건설기계,1097892.0,2009288.0,128.616149,38.078411
3230,2019,2019-12-31 12,주,화,1,1,0,0,1,경북,...,공작물충돌,안전운전 의무 불이행,교차로,교차로내,승용차,없음,1082569.9,1777238.5,128.415998,35.988434
3231,2019,2019-12-31 12,주,화,1,1,0,1,0,울산,...,횡단중,기타,교차로,교차로부근,승용차,보행자,1157722.0,1731868.0,129.240504,35.570292


In [3]:
data2 = data[data['사망자수']>1].copy()
data2

,발생년,발생년월일시,주야,요일,사망자수,부상자수,중상자수,경상자수,부상신고자수,발생지시도,...,사고유형,가해자법규위반,도로형태_대분류,도로형태,가해자_당사자종별,피해자_당사자종별,발생위치X_UTMK,발생위치Y_UTMK,경도,위도
77,2019,2019-01-11 17,주,금,2,0,0,0,0,제주,...,기타,안전운전 의무 불이행,단일로,기타단일로,승용차,보행자,893272.3,1477578.0,126.353839,33.284318
80,2019,2019-01-12 05,야,토,2,4,4,0,0,대구,...,측면충돌,신호위반,교차로,교차로내,승용차,승합차,1087747.0,1744593.0,128.469834,35.693687
103,2019,2019-01-14 05,야,월,2,2,2,0,0,경북,...,추돌,기타,단일로,고가도로위,화물차,화물차,1060789.0,1795019.0,128.175757,36.150331
113,2019,2019-01-15 02,야,화,2,0,0,0,0,경기,...,공작물충돌,안전운전 의무 불이행,단일로,기타단일로,화물차,없음,954298.0,1892940.0,126.986130,37.033887
207,2019,2019-01-24 20,야,목,3,5,4,1,0,강원,...,측면충돌,안전거리 미확보,단일로,지하차도(도로)내,화물차,승용차,1149446.5,1937500.2,129.189000,37.424600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3080,2019,2019-12-14 17,주,토,2,0,0,0,0,전남,...,기타,안전운전 의무 불이행,단일로,기타단일로,승용차,보행자,899665.0,1645458.0,126.403161,34.798744
3138,2019,2019-12-20 05,야,금,2,0,0,0,0,충북,...,보도통행중,안전운전 의무 불이행,교차로,교차로부근,화물차,보행자,998482.0,1853525.0,127.483010,36.679686
3153,2019,2019-12-21 19,야,토,2,3,0,3,0,경기,...,측면충돌,안전거리 미확보,단일로,고가도로위,화물차,승용차,936016.0,1938408.0,126.776676,37.442638
3186,2019,2019-12-26 14,주,목,2,0,0,0,0,전남,...,추돌,안전거리 미확보,단일로,기타단일로,사륜오토바이(ATV),특수차,1012636.0,1645371.0,127.638144,34.802824


In [4]:
data3 = pd.DataFrame(data['가해자법규위반'].value_counts()).copy()
data3.reset_index(level=0, inplace=True)
data3

np.savetxt("accident_type.txt", data3.values, 
           fmt='%s', delimiter="', ", newline="],\n['") 


In [5]:
data3

,index,가해자법규위반
0,안전운전 의무 불이행,2171
1,신호위반,304
2,중앙선 침범,232
3,과속,217
4,보행자 보호의무 위반,116
5,기타,92
6,안전거리 미확보,56
7,교차로 통행방법 위반,45


In [6]:
days = pd.DataFrame(data['요일'].value_counts()).copy()
days.reset_index(level=0, inplace=True)

np.savetxt("daily_accident.txt", days.values, 
           fmt='%s', delimiter="', ", newline="],\n['") 

days

,index,요일
0,금,517
1,월,507
2,목,482
3,수,482
4,화,464
5,토,429
6,일,352


In [7]:
March=len(data[data['발생년월일시'].str.contains('2019-03')])
April=len(data[data['발생년월일시'].str.contains('2019-04')])
May=len(data[data['발생년월일시'].str.contains('2019-05')])
June=len(data[data['발생년월일시'].str.contains('2019-06')])
July=len(data[data['발생년월일시'].str.contains('2019-07')])
August=len(data[data['발생년월일시'].str.contains('2019-08')])
September=len(data[data['발생년월일시'].str.contains('2019-09')])
October=len(data[data['발생년월일시'].str.contains('2019-10')])
November=len(data[data['발생년월일시'].str.contains('2019-11')])
December=len(data[data['발생년월일시'].str.contains('2019-12')])
January=len(data[data['발생년월일시'].str.contains('2019-01')])
Febuary=len(data[data['발생년월일시'].str.contains('2019-02')])

df = pd.DataFrame({'Month': ['3월','4월','5월','6월','7월','8월','9월','10월','11월','12월','01월','02월']
                  ,'counts': [March,April,May,June,July,August,September,October,November,December,January,Febuary]})
np.savetxt("Monthly_accident.txt", df.values, 
           fmt='%s', delimiter="', ", newline="],\n['") 

df

,Month,counts
0,3월,247
1,4월,280
2,5월,297
3,6월,268
4,7월,230
5,8월,241
6,9월,279
7,10월,320
8,11월,286
9,12월,298


In [8]:
korea = pd.read_csv('./datasets/Assignment4(traffic)/한국지역코드.CSV', encoding = 'cp949')
korea

,코드,이름
0,KR-11,서울
1,KR-26,부산
2,KR-27,대구
3,KR-28,인천
4,KR-29,광주
5,KR-30,대전
6,KR-31,울산
7,KR-50,세종
8,KR-41,경기
9,KR-42,강원


In [9]:
population = pd.read_csv('./datasets/Assignment4(traffic)/행정구역_시군구_별_인구수_20201128225046.csv', encoding = 'cp949')
population

,행정구역(시군구)별,총인구수 (명)
0,서울,9689159
1,부산,3397598
2,대구,2424171
3,인천,2942233
4,광주,1452994
5,대전,1467556
6,울산,1138581
7,세종,351007
8,경기,13400615
9,강원,1541836


In [10]:
area = pd.read_csv('./datasets/Assignment4(traffic)/지역별_면적_20201129011159.csv', encoding = 'cp949')
area

,지역,면적
0,서울,605
1,부산,770
2,대구,884
3,인천,1063
4,광주,501
5,대전,540
6,울산,1062
7,세종,465
8,경기,10188
9,강원,16828


1. 인구밀도 & 로그스케일

In [11]:
area_info = pd.merge(population, area, how='inner', left_on='행정구역(시군구)별', right_on='지역')
area_info = pd.merge(korea, area_info, how='inner', left_on='이름', right_on='지역')[['지역', '코드', '총인구수 (명)', '면적']]
area_info

,지역,코드,총인구수 (명),면적
0,서울,KR-11,9689159,605
1,부산,KR-26,3397598,770
2,대구,KR-27,2424171,884
3,인천,KR-28,2942233,1063
4,광주,KR-29,1452994,501
5,대전,KR-30,1467556,540
6,울산,KR-31,1138581,1062
7,세종,KR-50,351007,465
8,경기,KR-41,13400615,10188
9,강원,KR-42,1541836,16828


In [12]:
area_info.insert(area_info.shape[1], '인구밀도',area_info['총인구수 (명)']/area_info['면적'])
area_info['총인구수 (명)_로그스케일'] = preprocessing.scale(np.log(area_info['총인구수 (명)']+1))#총인구수 로그스케일
area_info['인구밀도_로그스케일'] = preprocessing.scale(np.log(area_info['인구밀도']+1)) # 인구밀도 로그스케일
area_info

,지역,코드,총인구수 (명),면적,인구밀도,총인구수 (명)_로그스케일,인구밀도_로그스케일
0,서울,KR-11,9689159,605,16015.138843,1.844797,2.112340
1,부산,KR-26,3397598,770,4412.464935,0.584153,1.213576
2,대구,KR-27,2424171,884,2742.274887,0.178054,0.882009
3,인천,KR-28,2942233,1063,2767.857949,0.411046,0.888481
4,광주,KR-29,1452994,501,2900.187625,-0.437704,0.921034
5,대전,KR-30,1467556,540,2717.696296,-0.425708,0.875733
6,울산,KR-31,1138581,1062,1072.110169,-0.731041,0.227537
7,세종,KR-50,351007,465,754.853763,-2.146618,-0.016842
8,경기,KR-41,13400615,10188,1315.333235,2.234914,0.369986
9,강원,KR-42,1541836,16828,91.623247,-0.366310,-1.480670


2. 지역별 교통사고 데이터 추가

In [13]:
# 지역에 코드 추가
new_data = pd.merge(data, korea, left_on='발생지시도', right_on='이름',  how='inner')
new_data = new_data.drop(['이름'], axis=1)

df = new_data.groupby(by=['코드']).sum().drop(['경도','위도','발생년','발생위치X_UTMK','발생위치Y_UTMK'], axis=1)
df

,사망자수,부상자수,중상자수,경상자수,부상신고자수
코드,,,,,
KR-11,250,89,33,48,8
KR-26,116,51,28,20,3
KR-27,112,81,24,53,4
KR-28,118,70,29,38,3
KR-29,49,15,5,10,0
KR-30,77,28,12,15,1
KR-31,55,19,7,10,2
KR-41,631,440,112,245,83
KR-42,185,101,62,36,3


In [14]:
# '교통사고 사망사건 인명피해자수' 추가
df.insert(df.shape[1], '교통사고 사망사건 인명피해자수', np.sum(df[['사망자수','부상자수','중상자수','경상자수','부상신고자수']], axis=1))
df.reset_index(level=0, inplace=True)
df

,코드,사망자수,부상자수,중상자수,경상자수,부상신고자수,교통사고 사망사건 인명피해자수
0,KR-11,250,89,33,48,8,428
1,KR-26,116,51,28,20,3,218
2,KR-27,112,81,24,53,4,274
3,KR-28,118,70,29,38,3,258
4,KR-29,49,15,5,10,0,79
5,KR-30,77,28,12,15,1,133
6,KR-31,55,19,7,10,2,93
7,KR-41,631,440,112,245,83,1511
8,KR-42,185,101,62,36,3,387
9,KR-43,188,105,48,55,2,398


In [15]:
df.corr()

,사망자수,부상자수,중상자수,경상자수,부상신고자수,교통사고 사망사건 인명피해자수
사망자수,1.000000,0.969366,0.940579,0.939654,0.802670,0.989900
부상자수,0.969366,1.000000,0.896346,0.989919,0.893932,0.994396
중상자수,0.940579,0.896346,1.000000,0.833109,0.624780,0.922179
경상자수,0.939654,0.989919,0.833109,1.000000,0.918859,0.975790
부상신고자수,0.802670,0.893932,0.624780,0.918859,1.000000,0.861430
교통사고 사망사건 인명피해자수,0.989900,0.994396,0.922179,0.975790,0.861430,1.000000


In [16]:
df['사망자수'].sum()

3349

In [17]:
# df(교통사고정보)와 area_info 병합
area_info_all = pd.merge(area_info, df, left_on='코드', right_on='코드')
area_info_all

,지역,코드,총인구수 (명),면적,인구밀도,총인구수 (명)_로그스케일,인구밀도_로그스케일,사망자수,부상자수,중상자수,경상자수,부상신고자수,교통사고 사망사건 인명피해자수
0,서울,KR-11,9689159,605,16015.138843,1.844797,2.112340,250,89,33,48,8,428
1,부산,KR-26,3397598,770,4412.464935,0.584153,1.213576,116,51,28,20,3,218
2,대구,KR-27,2424171,884,2742.274887,0.178054,0.882009,112,81,24,53,4,274
3,인천,KR-28,2942233,1063,2767.857949,0.411046,0.888481,118,70,29,38,3,258
4,광주,KR-29,1452994,501,2900.187625,-0.437704,0.921034,49,15,5,10,0,79
5,대전,KR-30,1467556,540,2717.696296,-0.425708,0.875733,77,28,12,15,1,133
6,울산,KR-31,1138581,1062,1072.110169,-0.731041,0.227537,55,19,7,10,2,93
7,세종,KR-50,351007,465,754.853763,-2.146618,-0.016842,15,1,1,0,0,17
8,경기,KR-41,13400615,10188,1315.333235,2.234914,0.369986,631,440,112,245,83,1511
9,강원,KR-42,1541836,16828,91.623247,-0.366310,-1.480670,185,101,62,36,3,387


In [18]:
area_info_all.insert(area_info_all.shape[1],'n*인명피해자수/총인구수',
                     area_info_all['교통사고 사망사건 인명피해자수']/area_info_all['총인구수 (명)']*370.844)

area_info_all.insert(area_info_all.shape[1],'n*인명피해자수/인구밀도',
                     area_info_all['교통사고 사망사건 인명피해자수']/area_info_all['인구밀도'])

area_info_all

,지역,코드,총인구수 (명),면적,인구밀도,총인구수 (명)_로그스케일,인구밀도_로그스케일,사망자수,부상자수,중상자수,경상자수,부상신고자수,교통사고 사망사건 인명피해자수,n*인명피해자수/총인구수,n*인명피해자수/인구밀도
0,서울,KR-11,9689159,605,16015.138843,1.844797,2.112340,250,89,33,48,8,428,0.016381,0.026725
1,부산,KR-26,3397598,770,4412.464935,0.584153,1.213576,116,51,28,20,3,218,0.023794,0.049405
2,대구,KR-27,2424171,884,2742.274887,0.178054,0.882009,112,81,24,53,4,274,0.041916,0.099917
3,인천,KR-28,2942233,1063,2767.857949,0.411046,0.888481,118,70,29,38,3,258,0.032519,0.093213
4,광주,KR-29,1452994,501,2900.187625,-0.437704,0.921034,49,15,5,10,0,79,0.020163,0.027240
5,대전,KR-30,1467556,540,2717.696296,-0.425708,0.875733,77,28,12,15,1,133,0.033608,0.048939
6,울산,KR-31,1138581,1062,1072.110169,-0.731041,0.227537,55,19,7,10,2,93,0.030291,0.086745
7,세종,KR-50,351007,465,754.853763,-2.146618,-0.016842,15,1,1,0,0,17,0.017961,0.022521
8,경기,KR-41,13400615,10188,1315.333235,2.234914,0.369986,631,440,112,245,83,1511,0.041815,1.148758
9,강원,KR-42,1541836,16828,91.623247,-0.366310,-1.480670,185,101,62,36,3,387,0.093082,4.223819


In [19]:
#지역별 인구

# bad visualization
np.savetxt("지역별_인구.txt", area_info_all[['코드','총인구수 (명)']].values,
           fmt='%s', delimiter="', ", newline="],\n['") 

# better visualization
np.savetxt("지역별_인구_로그스케일.txt", area_info_all[['코드','총인구수 (명)_로그스케일']].values,
           fmt='%s', delimiter="', ", newline="],\n['") 

In [20]:
#지역별 인구밀도

# bad visualizaion
np.savetxt("지역별_인구밀도.txt", area_info[['코드','인구밀도']].values,
           fmt='%s', delimiter="', ", newline="],\n['") 

# better visualizaion
np.savetxt("지역별_인구밀도_로그스케일.txt", area_info[['코드','인구밀도_로그스케일']].values,
           fmt='%s', delimiter="', ", newline="],\n['") 

# 맨 앞의 [' 는 직접 기입할 것
# 맨 끝의 [' 는 직접 지울 것

In [21]:
#지역별 교통사고수

np.savetxt("지역별_교통사고_사망사건_인명피해자수'.txt", area_info_all[['코드','교통사고 사망사건 인명피해자수']].values,
           fmt='%s', delimiter="', ", newline="],\n['") 
# 맨앞의 [' 는 직접 기입
# 맨 끝의 ,[' 는 직접 지우기

In [22]:
#인구비례 교통사고

np.savetxt("인구비례교통사고.txt", area_info_all[['코드','n*인명피해자수/총인구수']].values, 
           fmt='%s', delimiter="', ", newline="],\n['") 


#인구밀도비례 교통사고

np.savetxt("인구밀도비례교통사고.txt", area_info_all[['코드','n*인명피해자수/인구밀도']].values, 
           fmt='%s', delimiter="', ", newline="],\n['") 

In [23]:
#지역정보

np.savetxt("지역정보.txt", area_info_all[['코드','총인구수 (명)','면적']].values, 
           fmt='%s', delimiter="', ", newline="],\n['") 


In [24]:
region_data = pd.DataFrame(data['발생지시도'].value_counts())
region_data.reset_index(level=0, inplace=True)
region_data

,index,발생지시도
0,경기,608
1,경북,340
2,충남,299
3,경남,289
4,전남,277
5,서울,250
6,전북,228
7,충북,178
8,강원,172
9,인천,115


In [25]:
np.savetxt("지역별_사망사고_발생횟수.txt", region_data.values, 
           fmt='%s', delimiter="', ", newline="],\n['") 